In [1]:
import os
%pwd

'c:\\Users\\ojha.manish.kumar\\Documents\\Udemy\\DataScience_End_to_End_Projects\\data_science_mlops\\research'

In [2]:
os.chdir("../")
%pwd

'c:\\Users\\ojha.manish.kumar\\Documents\\Udemy\\DataScience_End_to_End_Projects\\data_science_mlops'

In [3]:
from dataclasses import dataclass
from pathlib import Path

In [4]:
@dataclass
class ModelTrainerConfig:
    root_dir:Path
    train_file_path:Path
    test_file_path:Path
    model_name:str
    penalty: str
    l1_ratio: float
    target_column:str


In [5]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self, 
        config_file_path=CONFIG_FILE_PATH,
        params_file_path=PARAM_FILE_PATH,
        schema_file_path=SCHEMA_FILE_PATH
        ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self)->ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.LogisticRegression
        schema = self.schema.TARGET_COLUMNS
        create_directories([config.root_dir])    

        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            train_file_path = config.train_file_path,
            test_file_path = config.test_file_path,
            model_name = config.model_name,
            penalty=params.penalty,
            l1_ratio=params.l1_ratio,
            target_column=schema.name
        )            

        return model_trainer_config



In [6]:
import pandas as pd
import os
from src.datascience import logger
from sklearn.linear_model import LogisticRegression
import joblib

In [7]:
class ModelTrainer:
    def __init__(self, config:ModelTrainerConfig):
        self.config = config

    def train_model(self):
        train_data = pd.read_csv(self.config.train_file_path)
        test_data  = pd.read_csv(self.config.test_file_path)

        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x  = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y  = test_data[[self.config.target_column]]

        lr = LogisticRegression(penalty=self.config.penalty, l1_ratio=self.config.l1_ratio)
        lr.fit(train_x, train_y)

        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))

In [8]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train_model()
except Exception as e:
     raise e

[2025-09-20 19:52:42,013: INFO: common]: yaml file: config\config.yaml loaded successfully
[2025-09-20 19:52:42,031: INFO: common]: yaml file: params.yaml loaded successfully


[2025-09-20 19:52:42,052: INFO: common]: yaml file: schema.yaml loaded successfully
[2025-09-20 19:52:42,054: INFO: common]: created directory at: artifacts
[2025-09-20 19:52:42,059: INFO: common]: created directory at: artifacts/model_trainer


c:\Users\ojha.manish.kumar\Documents\Udemy\DataScience_End_to_End_Projects\venv\lib\site-packages\sklearn\linear_model\_logistic.py:1221: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
c:\Users\ojha.manish.kumar\Documents\Udemy\DataScience_End_to_End_Projects\venv\lib\site-packages\sklearn\utils\validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\ojha.manish.kumar\Documents\Udemy\DataScience_End_to_End_Projects\venv\lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/module